# Introduction

In this notebook we will train Transformers-based models with TASS data. There are three classes "P", "Neu" and "N". We will use simpletransformers and wandb to train and log the results. 

| Negative | Neutral | Positive | Total |
|----------|---------|----------|-------|
| 22205 (41%)    | 3945 (7%)   | 27962 (51%)   | 54112 |



In [ ]:
!pip install --upgrade wandb
!pip install -q transformers
!pip install -q seqeval
!pip install -q tensorboardx
!pip install -U -q simpletransformers
!pip install -q tqdm

Requirement already up-to-date: wandb in /usr/local/lib/python3.6/dist-packages (0.10.12)


# Data

In [ ]:
import pandas as pd
TASS_data = pd.read_pickle("/content/drive/MyDrive/Datos_fake_news/Sentiment_Analysis/TASS/TASS_data_df.pkl")
text_data = TASS_data.content.to_list()
label_data = TASS_data.polarity.to_list()

# Split data

In [ ]:
from sklearn.model_selection import train_test_split

X_train_dev, X_test, y_train_dev, y_test = train_test_split(
    text_data, label_data, test_size=0.05, random_state=42, stratify =label_data)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train_dev, y_train_dev, test_size=0.02, random_state=42, stratify = y_train_dev)

In [ ]:
TASS_train_df = pd.DataFrame({"text": X_train, 
                              "labels": y_train})

TASS_dev_df = pd.DataFrame({"text": X_dev, 
                              "labels": y_dev})

TASS_test_df = pd.DataFrame({"text": X_test, 
                              "labels": y_test})

In [ ]:

TASS_train_df.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,N,20672
1,NEU,3673
2,P,26032


In [ ]:
TASS_dev_df.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,N,422
1,NEU,75
2,P,532


In [ ]:
TASS_test_df.groupby(["labels"], as_index=False).agg("count")

,labels,text
0,N,1111
1,NEU,197
2,P,1398


# TRAIN

In [ ]:
import logging

import pandas as pd
import sklearn

import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

# layer_parameters = {f"layer_{i}": {"values":5e-5} for i in range(0, 12, 1)}

sweep_config = {
    "name": "BETO",
    "method": "grid",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {  
                             "values" : [2] #10
        },
        "learning_rate": {
            "values":[ 6e-6] #  5e-5
        }, 
        'weight_decay': {
            'values': [0.0005, 0.005 ] # 0.05, 0.01
        },
        # "train_batch_size" : {"values":[16, 32]},
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,}
}

sweep_id = wandb.sweep(sweep_config, project="TASS-train")


# Logger
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Args
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.max_seq_length = 256
model_args.manual_seed = 4
model_args.learning_rate = 5e-5
model_args.use_multiprocessing = True
model_args.train_batch_size = 16 # Train = 63749 / 16 = 42010 batches in 1 epoch
model_args.eval_batch_size = 8
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training = True
model_args.best_model_dir = "outputs/best_model" # lower eval_loss
model_args.evaluate_during_training_steps = 1500
model_args.save_model_every_epoch = False 
model_args.save_eval_checkpoints = False # We dont have enough space
model_args.use_cached_eval_features = True
model_args.train_custom_parameters_only = False # Train specific parameters? No, train all parameters
model_args.gradient_accumulation_steps = 2 # The number of training steps (batches) to execute before performing a optimizer.step(). More = less time 
model_args.labels_list = ["N", "NEU", "P"]
# model_args.wandb_project = "SST2-train"

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
from sklearn.metrics import f1_score, accuracy_score,  matthews_corrcoef, cohen_kappa_score

def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        'bert', 
        "dccuchile/bert-base-spanish-wwm-uncased",
        use_cuda=False,
        args=model_args,
        num_labels=3, 
        sweep_config=wandb.config,
    )

    # Train the model and evalaute. Select best model according to evalaution
    model.train_model(TASS_train_df, 
                      eval_df=TASS_dev_df,  
                      acc=sklearn.metrics.accuracy_score,
                      f1 = f1_multiclass,
                      mcc = sklearn.metrics.matthews_corrcoef,
                      kappa = cohen_kappa_score
                      )

    # Evaluate the model
    # model.eval_model(sst2_df_test, acc=sklearn.metrics.accuracy_score, 
                    #  verbose=True)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

In [ ]:
import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

In [ ]:
import logging

import pandas as pd
import sklearn


sweep_config = {
    "method": "grid",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [1]},
        "learning_rate": {"values":[5e-5]},
    },
}

# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")
sweep_id = wandb.sweep(sweep_config, project="Hola")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
train_data = [
    ["Aragorn was the heir of Isildur", "true"],
    ["Frodo was the heir of Isildur", "false"],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_data = [
    ["Theoden was the king of Rohan", "true"],
    ["Merry was the king of Rohan", "false"],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.labels_list = ["true", "false"]
model_args.wandb_project = "Simple Sweep"

def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        use_cuda=False,
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(train_df, eval_df=eval_df)

    # Evaluate the model
    model.eval_model(eval_df)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None


Create sweep with ID: rlyl5w3p
Sweep URL: https://wandb.ai/huertas_97/Hola/sweeps/rlyl5w3p


wandb: Agent Starting Run: p0ba3z24 with config:
wandb: 	learning_rate: 5e-05
wandb: 	num_train_epochs: 1
wandb: Currently logged in as: huertas_97 (use `wandb login --relogin` to force relogin)


INFO:filelock:Lock 139764250354688 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock


...

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 1, 'fp': 0, 'fn': 1, 'eval_loss': 0.693641185760498}


global_step,1
tp,0
tn,1
fp,0
fn,1
mcc,0.0
train_loss,0.65265
eval_loss,0.69364
_step,3
_runtime,57
_timestamp,1609847120


global_step,▁
tp,▁
tn,▁
fp,▁
fn,▁
mcc,▁
train_loss,▁
eval_loss,▁
_step,▁▃▆█
_runtime,▁███
_timestamp,▁███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
distilmodel = ClassificationModel('roberta', '/content/outputs/best_model',
                                  use_cuda = False
                                          
                                          )

In [ ]:
!pip install httplib2==0.15.0

!pip install google-api-python-client==1.6

     |████████████████████████████████| 102kB 4.0MB/s 
ERROR: google-api-python-client 1.7.12 has requirement httplib2<1dev,>=0.17.0, but you'll have httplib2 0.15.0 which is incompatible.
  Found existing installation: httplib2 0.17.4
    Uninstalling httplib2-0.17.4:
      Successfully uninstalled httplib2-0.17.4


     |████████████████████████████████| 61kB 3.0MB/s 
ERROR: firebase-admin 4.4.0 has requirement google-api-python-client>=1.7.8, but you'll have google-api-python-client 1.6.0 which is incompatible.
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'model_prueba.zip'})
uploaded.SetContentFile('model_prueba.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1MJrpErQxnu8duKFM9_8wLrwDjxmkxDNl
